# 1-5. 출력 파서

In [1]:
!pip install -q langchain-google-genai


In [7]:
!pip install -q nbdev

In [3]:
# 구글 인증키 로딩
import os, dotenv
dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
# GEMINI_API_KEY # 이 출력 결과도 커밋이 되지 않도록 해야 함.
None

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 로드
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    # max_output_tokens=1000,
    google_api_key=GEMINI_API_KEY
)
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x117628e10>, default_metadata=(), model_kwargs={})

In [26]:
ChatGoogleGenerativeAI?

Init signature:
ChatGoogleGenerativeAI(
    *,
    name: Optional[str] = None,
    cache: Union[langchain_core.caches.BaseCache, bool, NoneType] = None,
    verbose: bool = <factory>,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    tags: Optional[list[str]] = None,
    metadata: Optional[dict[str, Any]] = None,
    custom_get_token_ids: Optional[Callable[[str], list[int]]] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    rate_limiter: Optional[langchain_core.rate_limiters.BaseRateLimiter] = None,
    disable_streaming: Union[bool, Literal['tool_calling']] = False,
    model: str,
    api_key: Optional[pydantic.types.SecretStr] = <factory>,
    credentials: Any = None,
    temperature: float = 0.7,
    top_p: Optional[float] = None,
    top_k: Optional[int] = None,
    max_tokens: Optional[int] = None,
    n: int = 1,
    max_retries

## 1-5-1. CSV Parser

https://wikidocs.net/231383

In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})
# ['Kimchi', 'Bibimbap', 'Korean BBQ', 'Bulgogi', 'Japchae']

['Kimchi', 'Bibimbap', 'Korean BBQ', 'Bulgogi', 'Japchae']

## 1-5-2. JSON Parser

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
# v1 warning
# /Users/yhlee/work/ai/ml_study/_venv/ml_gemini/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3670:
# LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally.
# The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used.
# Please update the code to import from Pydantic directly.

In [19]:
BaseModel?

Init signature: BaseModel() -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/work/ai/ml_study/_venv/ml_gemini/lib/python3.13/site-packages/pydantic/v1/main.py
Type:           ModelMetaclass
Subclasses:     BaseSettings, ExampleBase, Attachment, _AttachmentDict, ExampleCreate, AttachmentInfo, AttachmentsOperations, Attachment, ExampleUpdate, DatasetBase, ...

In [20]:
# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

In [21]:
# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [22]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [28]:
chain = prompt | llm | output_parser

# chain.invoke({"query": "Let me know how to cook Bibimbap"})
chain.invoke({"query": "비빔밥을 요리하는 방법을 500자 이내로 설명해줘"})

# 요리 방법 설명하는 도중이 글자가 잘린다.
# 최대 토큰 개수 설정을 해야 할 것 같음.

{'name': '비빔밥',
 'recipe': '비빔밥은 다양한 재료를 밥과 함께 비벼 먹는 한국의 대표적인 음식입니다.  만드는 방법은 다음과 같습니다.\n\n**1. 재료 준비:**\n* 밥: 2공기\n* 시금치: 1줌 (데쳐서 물기를 꼭 짜고, 참깨, 소금 간)\n* 무나물: 1/2개 (채 썰어 볶음)\n* 당근: 1/2개 (채 썰어 볶음)\n* 고사리: 1줌 (데쳐서 물기를 꼭 짜고, 간장, 참깨 간)\n* 버섯: 1줌 (볶음)\n* 콩나물: 1줌 (데쳐서 물기를 꼭 짜고, 간장, 참깨 간)\n* 소고기: 50g (불고기 양념으로 볶음)\n* 계란: 1개 (지단으로 부침)\n* 김: 1장 (잘게 부수기)\n* 참깨: 약간\n* 고추장: 1큰술\n* 간장: 1큰술\n* 참기름: 1큰술\n* 다진 마늘: 1작은술\n\n**2. 재료 조리:** 위에 나열된 재료들을 각각 맛있게 조리합니다.  야채는 데치거나 볶아 간을 맞추고, 고기는 불고기 양념으로 볶습니다. 계란은 지단으로 부쳐줍니다.\n\n**3. 비빔밥 만들기:**\n* 밥을 그릇에 담습니다.\n* 준비된 재료들을 밥 위에 보기 좋게 올립니다.\n* 고추장, 간장, 참기름, 다진 마늘을 넣고 잘 비벼 먹습니다.\n* 취향에 따라 김가루와 참깨를 뿌려 먹습니다.\n\n**팁:**  재료는 취향에 따라 추가하거나 변경할 수 있습니다.  나물의 간은 약간 심심하게 하는 것이 좋습니다.  비빔밥 양념장은 고추장, 간장, 참기름, 다진 마늘 외에 설탕이나 식초를 추가하여 입맛에 맞게 조절할 수 있습니다.'}